In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss
import seaborn as sns
import re

## 异常值全部替换为Nan，完成特征选择后再移含除异常值的sample，以最大程度保留样本

In [97]:
cancer_data = pd.read_csv('../../data/cancer_data_v4.csv')
cancer_data

,ID,"A=Case, B=Control",Origin,A=male B=female,Pattern,Age,Smoke,RBC,HGB,HCT,...,TG,TCH,HDL-C,LDL-C,ALP,GGT,CK,LDH-L,HBDH,T-CEA
0,1,A,A,B,A,41,0.0,3.78,122.0,0.37,...,0.53,3.86,1.99,1.67,47.0,11.0,38.0,156.0,118.0,NaN
1,100,A,A,A,A,35,0.0,5.53,169.0,0.52,...,1.23,6.60,2.49,3.85,43.0,30.0,23.0,191.0,159.0,NaN
2,1000,A,A,A,A,50,0.0,4.04,120.0,0.38,...,0.93,4.22,1.12,2.96,66.0,17.0,60.0,211.0,166.0,3.33
3,1007,A,A,B,A,50,NaN,4.14,131.0,0.38,...,1.76,3.97,1.47,2.20,58.0,13.0,64.0,227.0,173.0,0.76
4,1008,A,A,A,A,51,400.0,4.54,139.0,0.41,...,1.27,3.55,1.10,2.13,88.0,31.0,51.0,159.0,146.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7787,2489054,B,C,A,O,60,NaN,4.96,161.0,0.48,...,1.82,5.39,1.35,3.40,94.0,49.0,96.0,159.0,119.0,1.34
7788,2491728,B,C,A,O,48,0.0,5.35,174.0,0.51,...,0.97,5.22,1.11,3.71,60.0,65.0,76.0,164.0,129.0,0.97
7789,2491799,B,C,A,O,56,0.0,6.36,129.0,0.40,...,1.19,5.91,1.39,4.03,124.0,16.0,38.0,128.0,112.0,1.38
7790,509610,B,C,B,O,59,25.0,5.03,147.0,0.47,...,0.71,5.39,1.83,2.88,71.0,28.0,129.0,273.0,232.0,3.88
